In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [13]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import StudentPerformancePortugueseDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning2

from source.experiment_interface import run_exp_iter_with_LFR

## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'LFR_student_performance'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIR_INTERVENTION_PARAMS_LST = [{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp', 'LFR', EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                'LFR', EXPERIMENT_NAME, 'student_performance_por_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  638da338-6a90-44a7-a835-17f062b941fb


## Initialize custom objects

In [9]:
data_loader = StudentPerformancePortugueseDataset()
data_loader.X_data.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
0,18,4,4,2,2,0,4,3,4,1,...,course,mother,yes,no,no,no,yes,yes,no,no
1,17,1,1,1,2,0,5,3,3,1,...,course,father,no,yes,no,no,no,yes,yes,no
2,15,1,1,1,2,0,4,3,2,2,...,other,mother,yes,no,no,no,yes,yes,yes,no
3,15,4,2,1,3,0,3,2,2,1,...,home,mother,no,yes,no,yes,yes,yes,yes,yes
4,16,3,3,1,2,0,4,3,2,1,...,home,father,no,yes,no,no,yes,yes,no,no


In [10]:
data_loader.X_data.shape

(649, 32)

## Run experiment iterations

### Experiment iteration 1

In [11]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [14]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [15]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      with_tuning=True,
                      # with_tuning=False,
                      # tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='StudentPerformancePortugueseDataset',
                      verbose=True)

2024-01-02 14:15:14 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'model_init_seed': 100,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7997851699708123, L_x: 1.0313141050829486,  L_y: 0.6800812693613203,  L_z: 0.008286245050598566
step: 250, loss: 0.7997851032224239, L_x: 1.0313141423574097,  L_y: 0.680081199063262,  L_z: 0.008286244961710407
step: 500, loss: 0.7997851832253261, L_x: 1.0313141221279467,  L_y: 0.6800812828439846,  L_z: 0.008286244084273399
step: 750, loss: 0.6467933825057197, L_x: 1.0312323033840363,  L_y: 0.5272850592456668,  L_z: 0.008192546460824629
step: 1000, loss: 0.6467934160500395, L_x: 1.0312323140185529,  L_y: 0.5272850904234168,  L_z: 0.008192547112383732
step: 1250, loss: 0.5372168213232456, L_x: 1.029596052578625,  L_y: 0.4186896764543965,  L_z: 0.007783769805493282
step: 1500, loss: 0.5372168332076509, L_x: 1.0295960785432279,  L_y: 0.41868967710289384,  L_z: 0.00778377412521711
step: 1750, loss: 0.5336636887881964, L_x: 1.0284224064382381,  L_y: 0.41579636965404404,  L_z: 0.007512539245164261
step: 2000, l

2024-01-02 14:15:21 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__romantic_yes', 'num__age', 'num__Medu', 'nu

2024-01-02 14:16:09 experiment_interface.py INFO    : Models are tuned and saved to a file
INFO:root:Models are tuned and saved to a file


Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [16]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Student_Performance_Por_20240102__191609.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [17]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='StudentPerformancePortugueseDataset',
                      verbose=True)

2024-01-02 14:37:59 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 200,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7989742365933232, L_x: 1.0333738605431104,  L_y: 0.6789815789838769,  L_z: 0.008327635777567639
step: 250, loss: 0.7989741666340583, L_x: 1.0333738969712114,  L_y: 0.6789815068392818,  L_z: 0.008327635048827662
step: 500, loss: 0.7989742485844722, L_x: 1.0333738780784012,  L_y: 0.6789815929472827,  L_z: 0.00832763391467471
step: 750, loss: 0.6391934258825521, L_x: 1.0332909502036012,  L_y: 0.5194029750388243,  L_z: 0.00823067791168386
step: 1000, loss: 0.6391934637366692, L_x: 1.033290960107876,  L_y: 0.5194030101634384,  L_z: 0.008230678781221638
step: 1250, loss: 0.5228079191602457, L_x: 1.0314852347504928,  L_y: 0.403770397229017,  L_z: 0.007944499228089695
step: 1500, loss: 0.5228079383442957, L_x: 1.0314852594486816,  L_y: 0.4037704005509528,  L_z: 0.007944505924237373
step: 1750, loss: 0.5192984513523966, L_x: 1.0302606206214335,  L_y: 0.4009645894024503,  L_z: 0.007653899943901492
step: 2000, loss

2024-01-02 14:38:03 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 14:38:03 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__romantic_yes', 'num__age', 'num__Medu', 'nu

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [18]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Student_Performance_Por_20240102__191609.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [19]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='StudentPerformancePortugueseDataset',
                      verbose=True)

2024-01-02 15:03:44 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 300,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7979782676913761, L_x: 1.0323907659454599,  L_y: 0.6781985204253934,  L_z: 0.00827033533571835
step: 250, loss: 0.797978199748413, L_x: 1.0323908019191732,  L_y: 0.6781984500730722,  L_z: 0.008270334741711762
step: 500, loss: 0.7979782787217018, L_x: 1.0323907832154928,  L_y: 0.6781985335275081,  L_z: 0.00827033343632219
step: 750, loss: 0.6475774505477387, L_x: 1.0322746489837868,  L_y: 0.5280085073128342,  L_z: 0.008170739168262924
step: 1000, loss: 0.6475774873926531, L_x: 1.0322746589152256,  L_y: 0.5280085418657529,  L_z: 0.008170739817688817
step: 1250, loss: 0.5396801755981412, L_x: 1.0305529553397612,  L_y: 0.42116019193809506,  L_z: 0.007732344063034973
step: 1500, loss: 0.5396801895434374, L_x: 1.0305529807322427,  L_y: 0.42116019036751473,  L_z: 0.007732350551349179
step: 1750, loss: 0.5361632824016606, L_x: 1.0293364091548245,  L_y: 0.41834810283738577,  L_z: 0.007440769324396114
step: 2000, 

2024-01-02 15:03:48 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 15:03:48 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__romantic_yes', 'num__age', 'num__Medu', 'nu

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [20]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Student_Performance_Por_20240102__191609.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [ ]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='StudentPerformancePortugueseDataset',
                      verbose=True)

2024-01-02 15:29:16 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': "[{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}]",
 'intervention_param': "{'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}",
 'model_init_seed': 400,
 'session_uuid': '638da338-6a90-44a7-a835-17f062b941fb'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

intervention_options:  {'k': 10, 'Ax': 0.1, 'Ay': 1.0, 'Az': 2.0}
step: 0, loss: 0.7990623753426728, L_x: 1.0328282366486268,  L_y: 0.6788628044478547,  L_z: 0.00845837361497765
step: 250, loss: 0.799062310279977, L_x: 1.0328282720631976,  L_y: 0.6788627364890443,  L_z: 0.008458373292306452
step: 500, loss: 0.7990623867186859, L_x: 1.0328282538498756,  L_y: 0.6788628177803322,  L_z: 0.008458371776683092
step: 750, loss: 0.653315875429133, L_x: 1.0327228893027587,  L_y: 0.5333254837317553,  L_z: 0.008359051383550958
step: 1000, loss: 0.6533159097569131, L_x: 1.032722899285175,  L_y: 0.5333255157988329,  L_z: 0.008359052014781295
step: 1250, loss: 0.5497125316449264, L_x: 1.031007016982998,  L_y: 0.4307364335163241,  L_z: 0.00793769821515125
step: 1500, loss: 0.5497125427775167, L_x: 1.031007042805463,  L_y: 0.43073643312089216,  L_z: 0.007937702688039144
step: 1750, loss: 0.5462092990230252, L_x: 1.029785484764563,  L_y: 0.4278213886820478,  L_z: 0.007704680932260533
step: 2000, loss: 0

2024-01-02 15:29:19 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-02 15:29:19 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__romantic_yes', 'num__age', 'num__Medu', 'nu

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Student_Performance_Por_20240102__191609.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [ ]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='StudentPerformancePortugueseDataset',
                      verbose=True)

### Experiment iteration 6

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Student_Performance_Por_20240102__191609.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'LFR', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [ ]:
run_exp_iter_with_LFR(data_loader=exp_iter_data_loader,
                      experiment_seed=experiment_seed,
                      test_set_fraction=TEST_SET_FRACTION,
                      db_writer_func=db_writer_func,
                      fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                      models_params_for_tuning=models_params_for_tuning,
                      metrics_computation_config=metrics_computation_config,
                      custom_table_fields_dct=custom_table_fields_dct,
                      # with_tuning=True,
                      with_tuning=False,
                      tuned_params_df_paths=tuned_params_df_paths,
                      save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                      dataset_name='StudentPerformancePortugueseDataset',
                      verbose=True)